<a href="https://colab.research.google.com/github/ChrisTheDragon/Minha-Rede-Neural/blob/main/Aprendizado_com_TFF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Como usar Tensorflow-federated para construir uma rede com aprendizado federado**

## O que é `Tensorflow-federated`

A biblioteca TensorFlow Federated (TFF) é uma extensão do TensorFlow, que é um dos frameworks de aprendizado de máquina mais populares atualmente. O TFF foi projetado para permitir a construção e treinamento de modelos de aprendizado de máquina em um contexto de aprendizado federado. No aprendizado federado, em vez de reunir todos os dados em um local centralizado, o treinamento do modelo ocorre em dispositivos ou sistemas distribuídos, preservando a privacidade dos dados locais.

## Importações

In [ ]:
#@title
!pip install --upgrade tensorflow-federated

In [ ]:
import tensorflow as tf
import tensorflow_federated as tff

# Preparando os dados de entrada e pré processamento
A aprendizagem federada requer um conjunto de dados federados, ou seja, uma coleção de dados de vários usuários.

A fim de facilitar a experimentação, o repositório TFF tem alguns conjuntos de dados que podem ser usados livremente como teste, incluindo uma versão federado de MNIST que contém uma versão do conjunto de dados NIST originais. Que são digitos de 0-9 escritos a mão por escritores diferentes. Cada escritor será tratado como um cliente diferente.


In [ ]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

Para alimentar o conjunto de dados em nosso modelo, achatar os dados, e converter cada exemplo em uma tupla da forma (flattened_image_vector, label).

In [ ]:
NUM_CLIENTS = 10
BATCH_SIZE = 20

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch of EMNIST data and return a (features, label) tuple."""
    return (tf.reshape(element['pixels'], [-1, 784]), 
            tf.reshape(element['label'], [-1, 1]))

  return dataset.batch(BATCH_SIZE).map(batch_format_fn)

A função preprocess recebe um conjunto de dados, divide-o em lotes de tamanho fixo `BATCH_SIZE` e remodela os tensores de pixels e rótulos para uma forma adequada. Isso permite que os dados sejam processados em lotes durante o treinamento da rede neural no contexto do aprendizado federado.

In [ ]:
client_ids = sorted(emnist_train.client_ids)[:NUM_CLIENTS]
federated_train_data = [preprocess(emnist_train.create_tf_dataset_for_client(x))
  for x in client_ids
]

O código acima cria uma lista chamada `federated_train_data`, que contém os conjuntos de dados pré-processados para cada cliente (dispositivo) envolvido no aprendizado federado. 

Nota-se que o TFF trabalha com dois tipos de dados, os `tff.CLIENTS` e `tff.SERVER`, que são chamados de valores federados.

Basicamente para que o TFF reconheça os dados, eles devem passar por esse pre processamento em que serão transformados em `{float32}@CLIENTS` e quando processados pelo TFF retornaram como `{float32}@SERVER`.

Esses dois tipos de dados que serão transmitidos pela rede federada em questão. Assim mantendo a privacidade dos mesmos.

# Construindo o Modelo de Rede Neural
Este modelo (implementado através `tf.keras` ) tem uma única camada oculta, seguindo-se uma camada Softmax.
 
`Keras` é uma biblioteca de alto nível para construção e treinamento de redes neurais em Python que vem junto com o `tf`.

In [ ]:
def create_keras_model():
  initializer = tf.keras.initializers.GlorotNormal(seed=0)
  return tf.keras.models.Sequential([
      tf.keras.layers.Input(shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer=initializer),
      tf.keras.layers.Softmax(),
  ])

Utilizando o modelo criado acima, ele é convertido em um modelo federado que pode ser usado para treinamento em um contexto distribuído utilizando a função `tff.learning.from_keras_model`

In [ ]:
def model_fn():
  keras_model = create_keras_model()
  return tff.learning.models.from_keras_model(
      keras_model,
      input_spec=federated_train_data[0].element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Construindo o Algoritimo de Aprendizagem Federada
Em muitos casos, os algoritmos federados têm 4 componentes principais:
1. Uma etapa de transmissão de servidor para cliente.
2. Uma etapa de atualização do cliente local.
3. Uma etapa de upload de cliente para servidor.
4. Uma etapa de atualização do servidor.

Em TFF, um algoritimo federado é normalmente representado pela classe `tff.templates.IterativeProcess`. Esta é uma classe que contém as funções `initialize` e `next`. Aqui, `initialize` é usado para inicializar o servidor, e `next` realizará uma rodada de comunicação do algoritmo federado.

**O algoritimo autilizado aqui é um FedAvg para prever temperaturas em sensores** 

### Criando a Inicialização do Servidor

`server_init` cria uma `tff.learning.Model`, e retorna os seus pesos treináveis.

In [ ]:
@tff.tf_computation
def server_init():
  model = model_fn()
  return model.trainable_variables


`initializa_fn` transforma os pesos em tipos federados `{float32}@SERVER` que podem ser mandados aos servidores

In [ ]:
@tff.federated_computation
def initialize_fn():
  return tff.federated_value(server_init(), tff.SERVER)

Extrai os tipos do peso dos Servidores e Modelos para serem aplicados as funções

In [ ]:
whimsy_model = model_fn()
tf_dataset_type = tff.SequenceType(whimsy_model.input_spec)

In [ ]:
model_weights_type = server_init.type_signature.result

## Atualização do cliente local
Aqui se implementa a etapa de treinamento local de um cliente durante o treinamento federado. 
* Ele atribui os pesos do servidor ao modelo do cliente
* Executa um loop sobre os lotes de dados do cliente, 
* Calcula os gradientes dos pesos em relação à função de perda
* Aplica esses gradientes usando um otimizador específico do cliente. 
* Os pesos atualizados do cliente são retornados no final da função.

**Esse codigo vai no dispositivo**

In [ ]:
@tf.function
def client_update(model, dataset, server_weights, client_optimizer):
  """Performs training (using the server model weights) on the client's dataset."""
  # Initialize the client model with the current server weights.
  client_weights = model.trainable_variables
  # Assign the server weights to the client model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        client_weights, server_weights)

  # Use the client_optimizer to update the local model.
  for batch in dataset:
    with tf.GradientTape() as tape:
      # Compute a forward pass on the batch of data
      outputs = model.forward_pass(batch)

    # Compute the corresponding gradient
    grads = tape.gradient(outputs.loss, client_weights)
    grads_and_vars = zip(grads, client_weights)

    # Apply the gradient using a client optimizer.
    client_optimizer.apply_gradients(grads_and_vars)

  return client_weights

A função `client_update_fn` encapsula a lógica de treinamento local no cliente. Essa função cria um modelo local, configura um otimizador SGD e chama a função client_update para realizar o treinamento local usando o conjunto de dados federado e os pesos do modelo do servidor. A função de computação federada retorna o resultado do treinamento local como saída.

In [ ]:
@tff.tf_computation(tf_dataset_type, model_weights_type)
def client_update_fn(tf_dataset, server_weights):
  model = model_fn()
  client_optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
  return client_update(model, tf_dataset, server_weights, client_optimizer)

## Atualização do servidor.
A atualização do servidor para FedAvg é mais simples do que a atualização do cliente. Aqui apenas é substituido os pesos do modelo do servidor pela média dos pesos do modelo do cliente. Essa etapa de atualização ocorre após os clientes realizarem o treinamento local e enviarem as atualizações dos pesos para o servidor. A função retorna os pesos atualizados do modelo do servidor, que podem ser usados para a próxima iteração do treinamento federado.

In [ ]:
@tf.function
def server_update(model, mean_client_weights):
  """Updates the server model weights as the average of the client model weights."""
  model_weights = model.trainable_variables
  # Assign the mean client weights to the server model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        model_weights, mean_client_weights)
  return model_weights

Essa função cria um modelo global no servidor e chama a função 'server_update` para realizar a atualização dos pesos do modelo com base nos pesos médios dos clientes. A função de computação federada retorna os pesos atualizados do modelo do servidor como saída.

In [ ]:
@tff.tf_computation(model_weights_type)
def server_update_fn(mean_client_weights):
  model = model_fn()
  return server_update(model, mean_client_weights)

# Rodando o Modelo

Pega os pesos extraidos anteriormente e transforma em tipos federados

In [ ]:
federated_server_type = tff.FederatedType(model_weights_type, tff.SERVER)
federated_dataset_type = tff.FederatedType(tf_dataset_type, tff.CLIENTS)

A função `next_fn` recebe os pesos do servidor atual (`server_weights`) e um conjunto de dados federado (`federated_dataset`) como entrada e retorna os pesos atualizados do servidor após a iteração.

In [ ]:
@tff.federated_computation(federated_server_type, federated_dataset_type)
def next_fn(server_weights, federated_dataset):
  # Broadcast the server weights to the clients.
  server_weights_at_client = tff.federated_broadcast(server_weights)

  # Each client computes their updated weights.
  client_weights = tff.federated_map(
      client_update_fn, (federated_dataset, server_weights_at_client))

  # The server averages these updates.
  mean_client_weights = tff.federated_mean(client_weights)

  # The server updates its model.
  server_weights = tff.federated_map(server_update_fn, mean_client_weights)

  return server_weights

Função a ser chamada para o treinamento

In [ ]:
federated_algorithm = tff.templates.IterativeProcess(
    initialize_fn=initialize_fn,
    next_fn=next_fn
)

Cria um conjunto de dados para Teste

In [ ]:
central_emnist_test = emnist_test.create_tf_dataset_from_all_clients()
central_emnist_test = preprocess(central_emnist_test)

Essa função realiza uma avaliação do desempenho de um modelo Keras utilizando os pesos do servidor no aprendizado federado. Ela configura o modelo Keras para a avaliação, atribui os pesos do servidor ao modelo e calcula a perda e as métricas de desempenho usando um conjunto de teste. Isso permite avaliar quão bem o modelo está performando em dados não vistos durante o treinamento.

In [ ]:
def evaluate(server_state):
  keras_model = create_keras_model()
  keras_model.compile(
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]  
  )
  keras_model.set_weights(server_state)
  keras_model.evaluate(central_emnist_test)

# **Hora da Verdade**

## Avaliando o conjunto de teste

In [ ]:
server_state = federated_algorithm.initialize()
evaluate(server_state)

2042/2042 [==============================] - 18s 9ms/step - loss: 2.8479 - sparse_categorical_accuracy: 0.1027


## Treinando a Rede neural no servidor com 15 rodadas

In [ ]:
for round in range(15):
  server_state = federated_algorithm.next(server_state, federated_train_data)

In [ ]:
evaluate(server_state)

2042/2042 [==============================] - 16s 8ms/step - loss: 2.5867 - sparse_categorical_accuracy: 0.0980


## Depois de 100 rodadas

In [ ]:
for round in range(100):
  server_state = federated_algorithm.next(server_state, federated_train_data)

In [ ]:
evaluate(server_state)

2042/2042 [==============================] - 9s 4ms/step - loss: 1.7416 - sparse_categorical_accuracy: 0.3447


# Considerações Finais

Cada vez que o Servidor recebe os dados (Pesos) dos clientes, ele itera sobre os mesmos. 

Como podemos ver, a cada iteração a perda(loss) diminui e a precisão(accuracy) aumenta. 

Então esses pesos são retornados aos clientes.